In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://imslp.org/index.php?title=Category:Beethoven%2C%20Ludwig%20van&customcat=ccperson1"

html = requests.get(url).content

In [12]:
table = pd.read_html(html)[0]
table.head()

,Page Name,Cat. No.,I-Cat.,Key,Year
0,"Abbé Stadler, WoO 178 (Beethoven, Ludwig van)",WoO 178,ILB 1,B-flat major,1820
1,"Abendlied unter'm gestirnten Himmel, WoO 150 (...",WoO 150,ILB 2,E major,1820
2,"Abschiedsgesang an Wiens Bürger, WoO 121 (Beet...",WoO 121,ILB 3,G major,1796
3,"Abschiedsgesang, WoO 102 (Beethoven, Ludwig van)",WoO 102,ILB 4,B-flat major,1814
4,"Adagio for Mandolin and Harpsichord, WoO 43b (...",WoO 43b,ILB 5,E-flat major,1796


In [13]:
def get_url(s):
    return "https://imslp.org/wiki/" + s.replace(" ", "_")
    
table['links'] = table['Page Name'].apply(get_url)

table.head()

,Page Name,Cat. No.,I-Cat.,Key,Year,links
0,"Abbé Stadler, WoO 178 (Beethoven, Ludwig van)",WoO 178,ILB 1,B-flat major,1820,"https://imslp.org/wiki/Abbé_Stadler,_WoO_178_(..."
1,"Abendlied unter'm gestirnten Himmel, WoO 150 (...",WoO 150,ILB 2,E major,1820,https://imslp.org/wiki/Abendlied_unter'm_gesti...
2,"Abschiedsgesang an Wiens Bürger, WoO 121 (Beet...",WoO 121,ILB 3,G major,1796,https://imslp.org/wiki/Abschiedsgesang_an_Wien...
3,"Abschiedsgesang, WoO 102 (Beethoven, Ludwig van)",WoO 102,ILB 4,B-flat major,1814,"https://imslp.org/wiki/Abschiedsgesang,_WoO_10..."
4,"Adagio for Mandolin and Harpsichord, WoO 43b (...",WoO 43b,ILB 5,E-flat major,1796,https://imslp.org/wiki/Adagio_for_Mandolin_and...


In [15]:
print(table.links[20])

https://imslp.org/wiki/Andante_favori,_WoO_57_(Beethoven,_Ludwig_van)
